In [ ]:
! pip install textblob
! pip install nltk
! pip install 
! pip install torch
! pip install simpletransformers "transformers==4.30.2"
! pip install emoji

In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import nltk
from nltk.stem import WordNetLemmatizer
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')

In [38]:
data=pd.read_csv(r'C:\Users\Nurlan\Desktop\sentiment analiz\Combined Data.csv\Combined Data.csv',index_col=0)
data

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...
53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53040,Is there any way to sleep better? I can't slee...,Anxiety
53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [39]:
data.isnull().sum()

statement    362
status         0
dtype: int64

In [40]:
data.dropna(inplace=bool(1))

In [41]:
data['status'].value_counts()

Normal                  16343
Depression              15404
Suicidal                10652
Anxiety                  3841
Bipolar                  2777
Stress                   2587
Personality disorder     1077
Name: status, dtype: int64

In [42]:
# lower all words
data['statement']=data['statement'].str.lower()

In [43]:
# clear statements
data['statement']=data['statement'].str.replace('[^\w\s]','',regex=True)

In [44]:
# clear numbers
data['statement']=data['statement'].str.replace('\d+','',regex=True)

In [45]:
# clear links
data['statement']=data['statement'].str.replace('http\S+|www\S+|https\S+', '', flags=re.MULTILINE)

In [46]:
# stopwords
stop_words=set(stopwords.words('english'))
data['statement']=data['statement'].apply(lambda x:' '.join(word for word in x.split() if word not in stop_words))

In [47]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
data['statement']=data['statement'].apply(lambda x: ''.join(lemmatizer.lemmatize(word,pos='v') for word in x))

In [48]:
num_words = pd.Series(' '.join(data['statement']).split()).value_counts()
mini_num=num_words[num_words<2]
data['statement']=data['statement'].apply(lambda x:' '.join(i for i in x.split() if i not in mini_num))


In [49]:
data['status'].unique()

array(['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar',
       'Personality disorder'], dtype=object)

In [50]:
emotions={"Suicidal":0,"Depression":1,"Stress":2,"Normal":3,"Anxiety":4,"Bipolar":5,"Personality disorder":6}

In [51]:
data['status']=data['status'].map(emotions)

Tokenizer

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def encode_data(data, tokenizer, max_length=128):
    return tokenizer(
        data['statement'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

encoded_data = encode_data(data, tokenizer)


Train-Test datasetin hazırlanması

In [73]:
import torch
from sklearn.model_selection import train_test_split


labels = torch.tensor(data['status'].tolist())

train_texts, val_texts, train_labels, val_labels = train_test_split(
    encoded_data['input_ids'], labels, test_size=0.2, random_state=42
)

train_dataset = torch.utils.data.TensorDataset(train_texts, train_labels)
val_dataset = torch.utils.data.TensorDataset(val_texts, val_labels)


Modelin Qurulması

In [ ]:
from transformers import DistilBertForSequenceClassification

model =DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=7  
)


Model Training

In [75]:
from transformers import AdamW
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

optimizer = AdamW(model.parameters(), lr=5e-5)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):  
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} tamamlandı. Loss: {loss.item()}")


Epoch 1 tamamlandı. Loss: 0.5481103658676147
Epoch 2 tamamlandı. Loss: 0.6623845100402832
Epoch 3 tamamlandı. Loss: 0.08853846788406372
Epoch 4 tamamlandı. Loss: 0.026663903146982193
Epoch 5 tamamlandı. Loss: 0.04325753450393677
Epoch 6 tamamlandı. Loss: 0.07868964970111847
Epoch 7 tamamlandı. Loss: 0.018099820241332054
Epoch 8 tamamlandı. Loss: 0.07686415314674377
Epoch 9 tamamlandı. Loss: 0.04810839891433716
Epoch 10 tamamlandı. Loss: 0.002896461170166731


Modelin qiymətləndirilməsi

In [76]:
from sklearn.metrics import classification_report

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch
        inputs = inputs.to(device)
        outputs = model(inputs)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions))


              precision    recall  f1-score   support

           0       0.65      0.62      0.64      2158
           1       0.71      0.71      0.71      3016
           2       0.62      0.66      0.64       536
           3       0.93      0.91      0.92      3308
           4       0.83      0.80      0.82       755
           5       0.74      0.88      0.80       527
           6       0.71      0.76      0.74       237

    accuracy                           0.77     10537
   macro avg       0.74      0.76      0.75     10537
weighted avg       0.77      0.77      0.77     10537



Modelin Yoxlanması (Prediction)

In [81]:


tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

new_texts = ['Living with bipolar disorder means navigating extreme emotional highs and lows.']

encoded_new_texts = tokenizer(
    new_texts,
    padding="max_length",  
    truncation=True,       
    max_length=128,        
    return_tensors="pt"   
)

input_ids = encoded_new_texts["input_ids"]
attention_mask = encoded_new_texts["attention_mask"]


model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


with torch.no_grad():
    inputs = input_ids.to(device)
    masks = attention_mask.to(device)
    
    outputs = model(input_ids=inputs, attention_mask=masks)
    logits = outputs.logits  
    predictions = torch.argmax(logits, dim=1)  


labels = ["0", "1", "2", "3", "4", "5", "6"]

for text, pred in zip(new_texts, predictions.cpu().numpy()):
    print(f"Mətn: {text} | Proqnoz: {labels[pred]}")


Mətn: Living with bipolar disorder means navigating extreme emotional highs and lows. | Proqnoz: 5


In [61]:
emotions

{'Suicidal': 0,
 'Depression': 1,
 'Stress': 2,
 'Normal': 3,
 'Anxiety': 4,
 'Bipolar': 5,
 'Personality disorder': 6}